In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import sys
import os


In [2]:
import sys
import os
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

from config import LLM_MODEL

In [3]:
llm = ChatOpenAI(temperature=0.2, model=LLM_MODEL)

# Create recommendation prompt
recommendation_template = """
You are a financial advisor specializing in short-term stock investments.

Based on the following scores and insights for {ticker} ({company_name}), 
provide a detailed investment recommendation.

SCORES:
{scores}

RECENT INSIGHTS:
{key_insights}

Provide a comprehensive recommendation that includes:
1. Investment thesis (2-3 paragraphs)
2. Key supporting factors (bullet points)
3. Potential risks (bullet points)
4. Suggested entry points or conditions
5. Suggested exit strategy or price targets
6. Timeline considerations

Keep your recommendation focused on short-term investment horizon (last 3 months).
"""



In [4]:
recommendation_prompt = PromptTemplate(
    input_variables=["ticker", "company_name", "scores", "key_insights"],
    template=recommendation_template
)

recommendation_chain = recommendation_prompt | llm


In [5]:
from agents.research import ResearchAgent
from agents.filtering import FilteringSystem
from agents.extraction import ExtractionAgent
from agents.scoring import ScoringMechanism

research_agent = ResearchAgent()
filtering_system = FilteringSystem()
extraction_agent = ExtractionAgent()
scoring_mechanism = ScoringMechanism()

research_results = research_agent.research("MSFT", "Microsoft Corporation")
filtered_results = filtering_system.filter(research_results, "MSFT", "Microsoft Corporation")
extraction_results = extraction_agent.process(filtered_results)
scoring_results = scoring_mechanism.score(extraction_results)




> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Microsoft Corporation MSFT recent quarterly results earnings analyst ratings business developments market trends competitive landscape'}`


[{'title': 'Microsoft Corporation (MSFT) Analyst Ratings, Estimates & Forecasts', 'url': 'https://finance.yahoo.com/quote/MSFT/analysis/', 'content': '| Currency in USD | Current Qtr. (Mar 2025) | Next Qtr. (Jun 2025) | Current Year (2025) | Next Year (2026) |\n| --- | --- | --- | --- | --- |\n| No. of Analysts | 32 | 31 | 55 | 54 |\n| Avg. Estimate | 68.46B | 72.68B | 277.27B | 315.96B |\n| Low Estimate | 67.14B | 71.47B | 274.86B | 304.21B |\n| High Estimate | 69.36B | 74.63B | 283.15B | 341.76B |\n| Year Ago Sales | 61.86B | 64.73B | 245.12B | 277.27B |\n| Sales Growth (year/est) | 10.67% | 12.29% | 13.11% | 13.95% |\nEarnings History [...] Analyst Price Targets\n420.00\n506.80 Average\n409.75 Current\n650.00 High\nEarnings Estimate\n| Currency in US

In [6]:
ticker = scoring_results["ticker"]
company_name = scoring_results["company_name"]
score = scoring_results["score"]
extracted_insights = scoring_results["extracted_insights"]


In [8]:
extracted_insights

[{'url': 'https://finance.yahoo.com/quote/MSFT/)',
  'structured_insights': {'financial_metrics': [],
   'analyst_opinions': [],
   'business_developments': [],
   'market_sentiment': {'overall_sentiment': 'neutral',
    'trading_volume': '',
    'price_movement': '',
    'volatility': ''},
   'risk_factors': []},
  'summary': "Based on the provided article about Microsoft Corporation (MSFT), here are the key investment insights for a short-term investor (3-month horizon):\n\n1. **Recent Financial Results**: The article does not provide specific financial results, but investors should look for Microsoft's latest earnings report, focusing on revenue growth, profit margins, and guidance for future quarters.\n\n2. **Analyst Ratings and Price Targets**: The article lacks specific analyst ratings and price targets. Investors should check recent analyst reports for any upgrades or downgrades, as well as consensus price targets that could indicate potential upside or downside.\n\n3. **Busines

In [9]:

# Format scores
scores_text = f"""
Financial Health: {score.financial_health_score}/10 - {score.reasoning.get('financial_health_score', '')}
Growth Potential: {score.growth_potential_score}/10 - {score.reasoning.get('growth_potential_score', '')}
Analyst Sentiment: {score.analyst_sentiment_score}/10 - {score.reasoning.get('analyst_sentiment_score', '')}
Momentum: {score.momentum_score}/10 - {score.reasoning.get('momentum_score', '')}
Risk Level: {score.risk_score}/10 - {score.reasoning.get('risk_score', '')}

OVERALL SCORE: {score.overall_score}/100
RECOMMENDATION: {score.investment_recommendation}
CONFIDENCE: {score.confidence_level}
"""


In [10]:

# Extract key insights from each article
key_insights = ""
for i, insight in enumerate(extracted_insights[:3], 1):  # Use up to 3 articles
    key_insights += f"INSIGHT {i}:\n{insight['summary']}\n\n"
key_insights

"INSIGHT 1:\nBased on the provided article about Microsoft Corporation (MSFT), here are the key investment insights for a short-term investor (3-month horizon):\n\n1. **Recent Financial Results**: The article does not provide specific financial results, but investors should look for Microsoft's latest earnings report, focusing on revenue growth, profit margins, and guidance for future quarters.\n\n2. **Analyst Ratings and Price Targets**: The article lacks specific analyst ratings and price targets. Investors should check recent analyst reports for any upgrades or downgrades, as well as consensus price targets that could indicate potential upside or downside.\n\n3. **Business Developments and News**: The article does not mention any recent business developments or news. Investors should stay updated on any product launches, partnerships, or acquisitions that could impact Microsoft's market position and stock performance.\n\n4. **Market Sentiment and Trading Patterns**: The article does

In [19]:

# Generate recommendation
result = recommendation_chain.invoke({
    "ticker": ticker,
    "company_name": company_name,
    "scores": scores_text,
    "key_insights": key_insights
})
result

AIMessage(content="### Investment Recommendation for Microsoft Corporation (MSFT)\n\n#### Investment Thesis\nMicrosoft Corporation (MSFT) presents a compelling short-term investment opportunity driven by its robust financial health, significant growth potential, and positive market momentum. The company has demonstrated impressive revenue growth of 12% year-over-year, with notable contributions from its AI and cloud services segments, which are expanding at rates of 175% and 21%, respectively. This strong performance not only underscores Microsoft's competitive positioning in the tech sector but also reflects its ability to innovate and adapt to changing market demands. The overall positive sentiment among analysts, despite some risks, supports a favorable outlook for MSFT in the near term.\n\nIn addition to its strong financial results, Microsoft is strategically investing in its cloud and AI infrastructure, which positions it well for sustained growth. The company's focus on operatio

In [20]:

recommendation_results = {
    "ticker": ticker,
    "company_name": company_name,
    "recommendation": result.content,  
    "score": score
}


print(f"Recommendation for {recommendation_results['ticker']}:")
print(recommendation_results['recommendation'])

Recommendation for MSFT:
### Investment Recommendation for Microsoft Corporation (MSFT)

#### Investment Thesis
Microsoft Corporation (MSFT) presents a compelling short-term investment opportunity driven by its robust financial health, significant growth potential, and positive market momentum. The company has demonstrated impressive revenue growth of 12% year-over-year, with notable contributions from its AI and cloud services segments, which are expanding at rates of 175% and 21%, respectively. This strong performance not only underscores Microsoft's competitive positioning in the tech sector but also reflects its ability to innovate and adapt to changing market demands. The overall positive sentiment among analysts, despite some risks, supports a favorable outlook for MSFT in the near term.

In addition to its strong financial results, Microsoft is strategically investing in its cloud and AI infrastructure, which positions it well for sustained growth. The company's focus on operati

In [23]:
recommendation_results['score'].overall_score

76